# data는 김도휘 형제님과 김명찬 형제님이 만들어주신 보편지향 기도 데이터를 사용하였습니다. 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

## CSV 에서 기도문 읽어오기
def read_data(path_to_file):
    df = pd.read_csv(path_to_file, dtype=str)
    return df

df = read_data('../../data/pray1.csv')

In [2]:
df.to_csv('../../data/pray1_withid.csv')

In [180]:
X = df['content']
y = df['label']
print(len(X))
print(len(y))

274
274


In [181]:
X[0]

'주님, 구원의 말씀을 전하는 교회가 주님의 말씀안에서 생활하며 많은 이들을 주님의 품으로 인도하게 하소서.'

In [209]:
y_quiz = df['content'].sample(100)
y_quiz.shape

(100,)

In [212]:
y_quiz.sort_index().to_csv('../../data/quiz_pray1_sample100.csv')

/Users/mckim/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


## y data one_hot encoding

In [182]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

print(type(y[0]), y[:5])

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
print(integer_encoded[:5])

# one_hot encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded[:5])

# setup y 
# y = onehot_encoded
y= integer_encoded
print(y[:5, :])

<class 'str'> 0    1
1    1
2    1
3    1
4    1
Name: label, dtype: object
[0 0 0 0 0]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
[[0]
 [0]
 [0]
 [0]
 [0]]


/Users/mckim/anaconda3/envs/py3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## 띄어쓰기로 구분

In [183]:
X = [x.split() for x in X]

In [184]:
X[0]

['주님,',
 '구원의',
 '말씀을',
 '전하는',
 '교회가',
 '주님의',
 '말씀안에서',
 '생활하며',
 '많은',
 '이들을',
 '주님의',
 '품으로',
 '인도하게',
 '하소서.']

## 고유 토큰 인덱싱

In [185]:
from collections import defaultdict

In [186]:
# 단어마다 고유한 인덱스를 부여하기 위한 dictionary
token_to_index = defaultdict(lambda : len(token_to_index))

In [187]:
# 단어에 대한 고유 인덱스를 부여하는 함수
def convert_token_to_idx(token_ls):
    for tokens in token_ls:
        yield [token_to_index[token] for token in tokens]
    return

In [188]:
X = list(convert_token_to_idx(X))

In [189]:
# 고유 인덱스로 변환될 경우, 원래 어떤 단어였는지 알기 어려우므로,
# 인덱스로 변환된 단어를 본래의 단어로 재변환하기 위한 dictionary 생성
index_to_token = {val : key for key,val in token_to_index.items()}

#### 인덱싱 결과 확인 

In [190]:
import operator

In [191]:
for k,v in sorted(token_to_index.items(), key=operator.itemgetter(1))[:5]:
    print (k,v)

주님, 0
구원의 1
말씀을 2
전하는 3
교회가 4


In [192]:
X[0]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 5, 10, 11, 12]

### 빈(empty) 단어 가방(Bag of Words) 생성

In [193]:
n_train_reviews = len(X)       # 학습용 리뷰의 총 수
n_unique_word = len(token_to_index)  # 고유 단어의 갯수 (BOW의 차원의 크기) 

In [194]:
n_unique_word

2048

### numpy를 사용하면 memory error 발생 

In [195]:
import numpy as np

In [196]:
bow = np.zeros((n_train_reviews, n_unique_word), dtype=np.int8)

### Scipy 패키지 활용

In [197]:
import scipy.sparse as sps

In [198]:
# 학습용 리뷰 수(150,000) x 고유 단어의 수(450,541)의 크기를 갖는 빈 단어가방 생성
bow_data = sps.lil_matrix((n_train_reviews, n_unique_word), dtype=np.int8)

### 단어 가방 채우기

In [199]:
for i, tokens in enumerate(X):
    for token in tokens:
        # i번 째 리뷰에 등장한 단어들을 세서, 고유 번호에 1씩 더해준다.
        bow_data[i, token] += 1

### Train / test split

In [200]:
bow_train, bow_test, y_train, y_test = train_test_split(bow_data, y, test_size=0.2, random_state=1212)
print(bow_train.shape, bow_test.shape, y_train.shape, y_test.shape)

(219, 2048) (55, 2048) (219, 1) (55, 1)


## Logistic Regression

In [201]:
from sklearn.linear_model import LogisticRegression

In [204]:
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

## Train

In [205]:
model.fit(bow_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# Test

In [206]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [207]:
predict = model.predict(bow_test)
accuracy = accuracy_score(y_test, predict)

In [208]:
print('Accuracy : ',accuracy)
print(classification_report(y_test, predict))

Accuracy :  0.7454545454545455
              precision    recall  f1-score   support

           0       0.67      1.00      0.80        16
           1       0.78      0.74      0.76        19
           2       0.89      0.73      0.80        11
           3       0.75      0.33      0.46         9

    accuracy                           0.75        55
   macro avg       0.77      0.70      0.70        55
weighted avg       0.76      0.75      0.73        55

